In [6]:
import numpy as np 
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [7]:
stocks = pd.read_csv('sp_500_stocks-2.csv')
from secrets import IEX_CLOUD_API_TOKEN


In [11]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2183338686310,
 'week52high': 178.58,
 'week52low': 124.17,
 'week52highSplitAdjustOnly': 179.61,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': -0.14509457162646566,
 'sharesOutstanding': 15836213000,
 'float': 0,
 'avg10Volume': 65926637,
 'avg30Volume': 75428907,
 'day200MovingAvg': 148.09,
 'day50MovingAvg': 134.68,
 'employees': 147000,
 'ttmEPS': 6.11,
 'ttmDividendRate': 0.906784243631777,
 'dividendYield': 0.0065770961313685134,
 'nextDividendDate': '',
 'exDividendDate': '2022-11-04',
 'nextEarningsDate': '2023-02-02',
 'peRatio': 21.876483535665265,
 'beta': 1.3029490514086624,
 'maxChangePercent': 52.44627073964956,
 'year5ChangePercent': 2.25388415162293,
 'year2ChangePercent': 0.020684651603541715,
 'year1ChangePercent': -0.14509457162646566,
 'ytdChangePercent': 0.06110982836912182,
 'month6ChangePercent': -0.10868922947227956,
 'month3ChangePercent': -0.06109201257416197,
 'month1ChangePercent': 0.01915150296571833,
 'day

In [ ]:
data['year1ChangePercent']

In [32]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [34]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
    
final_dataframe

https://cloud.iexapis.com/stable/stock/market/batch?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=price,stats&token=sk_ba1cf67897964954bb4191e85ea1f6f4


TypeError: 'Response' object is not subscriptable

In [20]:
#remove low momentum stocks
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [ ]:
#calculate the number of shares to buy
def portfolio_size():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
            


In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

In [27]:
#better momentum strategy
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
    
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['Price'],
                'Number of Shares to Buy',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A'

            ],
            index = hqm_columns),
            ignore_index = True
        
        
        
        )

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#calculate momentum percentile
time_periods = [
                'One_Year',
                'Six-month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])

hqm_dataframe


In [29]:
from statistics import mean

for row in hqm_dataframe.index:
    momentun_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[])

4